# Loading all 13 lat-lon-cap NetCDF tile files at once

## Objectives

Introduce an easy method for loading all 13 llc NetCDF tile files at once.  

## Loading all 13 lat-lon-cap NetCDF grid tile files using  `load_all_tiles_from_netcdf`

So far we've loaded NetCDF tile files one tile at a time.  The `load_all_tiles_from_netcdf` method automates the loading of all 13 tiles.

Let's jump right in and use `load_all_tiles_from_netcdf` to load all 13 GRID tile files.  Call the new `Dataset` object `grid_all_tiles`.   Because we are loading GRID tile files, we specify 'grid' as the *var_type*.

In [1]:
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import ecco_v4_py as ecco

# Set this to be the directory for your fields
ECCO_dir = '/Users/ifenty/ECCOv4/R3'

# Load all tiles of the LLC90 Grid    
grid_dir= ECCO_dir + '/nctiles_grid/'  
var = 'GRID'
var_type = 'grid'
grid_all_tiles = ecco.load_all_tiles_from_netcdf(grid_dir, var, var_type)

# minimize the metadata (optional)
ecco.minimal_metadata(grid_all_tiles)


>>> LOADING TILES FROM NETCDF

loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0001.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0002.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0003.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0004.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0005.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0006.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0007.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0008.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0009.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0010.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0011.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0012.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_grid/GRID.0013.nc
total file load time  0.734987974167 s
concatenated all tiles.  this can take a few minutes....
finished concatenating.  time = 0.689743995667 s
Finished loading all 13 tiles of GRID
Removing Dataset Attributes A-Z

Let's look at `grid_all_tiles`:

In [2]:
grid_all_tiles

<xarray.Dataset>
Dimensions:  (i: 90, i_g: 90, j: 90, j_g: 90, k: 50, k_g: 50, tile: 13)
Coordinates:
  * k        (k) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * i        (i) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * j        (j) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * i_g      (i_g) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * j_g      (j_g) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * k_g      (k_g) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * tile     (tile) int64 1 2 3 4 5 6 7 8 9 10 11 12 13
Data variables:
    XC       (tile, j, i) float64 -111.6 -111.3 -110.9 -110.5 -110.0 -109.3 ...
    YC       (tile, j, i) float64 -88.24 -88.38 -88.52 -88.66 -88.8 -88.94 ...
    RAC      (tile, j, i) float64 3.623e+08 3.633e+08 3.643e+08 3.651e+08 ...
    Depth    (tile, j, i) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    AngleCS  (tile, j, i) float64 0.0615

### Examining the Dataset object contents

#### 1. Dimensions
`Dimensions:  (i: 90, i_g: 90, j: 90, j_g: 90, k: 50, k_g: 50, tile: 13)`

The *Dimensions* list now inculdes a new **tile** dimension.

#### 2. Coordinates
```
Coordinates:
  * k        (k) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * i        (i) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * j        (j) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * i_g      (i_g) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * j_g      (j_g) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * k_g      (k_g) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * tile     (tile) int64 1 2 3 4 5 6 7 8 9 10 11 12 13
``` 

The **tile** coordinate now appears as an array: 1 .. 13


#### 3. Data Variables
```
Data variables:
    XC       (tile, j, i) float64 -111.6 -111.3 -110.9 -110.5 -110.0 -109.3 ...
    hFacC    (tile, k, j, i) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    XG       (tile, j_g, i_g) float64 -115.0 -115.0 -115.0 -115.0 -115.0 ...
    DXC      (tile, j, i_g) float64 1.558e+04 1.559e+04 1.559e+04 1.56e+04 ...
    hFacW    (tile, k, j, i_g) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    DYC      (tile, j_g, i) float64 1.156e+04 1.159e+04 1.162e+04 1.165e+04 ...
    hFacS    (tile, k, j_g, i) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    RC       (k) float64 -5.0 -15.0 -25.0 -35.0 -45.0 -55.0 -65.0 -75.0 ...
    RF       (k_g) float64 0.0 -10.0 -20.0 -30.0 -40.0 -50.0 -60.0 -70.0 ...
```

Each *Data variable* that varies in the horizontal now has a new dimension, **tile**. 

> **Note:** The ordering of the arrays is (tile, k, j, i)

### A closer look at the DataArray after loading with `load_all_tiles_from_netcdf`

Let's look at one of the grid variables loaded with the `load_all_tiles_from_netcdf` routine:

In [3]:
grid_all_tiles.XC

<xarray.DataArray 'XC' (tile: 13, j: 90, i: 90)>
array([[[-111.606468, -111.303001, ...,   64.805206,   64.819168],
        [-104.819603, -103.928444, ...,   64.410118,   64.4524  ],
        ...,
        [ -37.5     ,  -36.5     , ...,   50.5     ,   51.5     ],
        [ -37.5     ,  -36.5     , ...,   50.5     ,   51.5     ]],

       [[ -37.5     ,  -36.5     , ...,   50.5     ,   51.5     ],
        [ -37.5     ,  -36.5     , ...,   50.5     ,   51.5     ],
        ...,
        [ -37.5     ,  -36.5     , ...,   50.5     ,   51.5     ],
        [ -37.5     ,  -36.5     , ...,   50.5     ,   51.5     ]],

       ...,

       [[-127.5     , -127.5     , ..., -127.5     , -127.5     ],
        [-126.5     , -126.5     , ..., -126.5     , -126.5     ],
        ...,
        [ -39.5     ,  -39.5     , ...,  -39.5     ,  -39.5     ],
        [ -38.5     ,  -38.5     , ...,  -38.5     ,  -38.5     ]],

       [[-127.5     , -127.5     , ..., -115.505669, -115.166985],
        [-126.5     , 

The $XC$ `DataArray` object now has a **tile** dimension of length 13.

## Loading non-grid variables using `load_all_tiles_from_netcdf`

As with `load_tile_from_netcdf` we can use `load_all_tiles_from_netcdf` to give variables better coordinate labels by specifying their c-grid point category.

We'll demonstrate by loading the ``SSH`` variable that is on c-grid $c$ points.  Loading variables on $u$, $v$, or $g$ points only requires changing the argument of **var_type**.

### A *c* point variable: SSH

Let's load all tiles of sea surface height (SSH), a $c$ point variable.  

In [4]:
# Load all tiles of the LLC90 Grid    
data_dir= ECCO_dir + '/nctiles_monthly/SSH/'

var = 'SSH'
var_type = 'c'
ssh_all_tiles = ecco.load_all_tiles_from_netcdf(data_dir, var, var_type)
ecco.minimal_metadata(ssh_all_tiles)


>>> LOADING TILES FROM NETCDF

loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0001.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0002.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0003.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0004.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0005.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0006.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0007.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0008.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0009.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0010.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0011.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0012.nc
loading /Users/ifenty/ECCOv4/R3/nctiles_monthly/SSH/SSH.0013.nc
total file load time  0.687794923782 s
concatenated all tiles.  this can take a few minutes....
finished concatenating.  time = 0.348370

### Examining the Dataset object contents

Let's examine $ssh_all_tiles$ since this is first time we're loading all 13 tiles of an output variable.

In [5]:
ssh_all_tiles

<xarray.Dataset>
Dimensions:   (i: 90, j: 90, tile: 13, time: 288)
Coordinates:
  * time      (time) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * j         (j) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * i         (i) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
    tim       (time) datetime64[ns] 1992-01-16 1992-02-16 1992-03-16 ...
    timestep  (time) float64 732.0 1.428e+03 2.172e+03 2.892e+03 3.636e+03 ...
    lon_c     (tile, j, i) float64 -111.6 -111.3 -110.9 -110.5 -110.0 -109.3 ...
    lat_c     (tile, j, i) float64 -88.24 -88.38 -88.52 -88.66 -88.8 -88.94 ...
  * tile      (tile) int64 1 2 3 4 5 6 7 8 9 10 11 12 13
Data variables:
    SSH       (time, tile, j, i) float64 nan nan nan nan nan nan nan nan nan ...
Attributes:
    description:    SSH -- ECCO v4 ocean state estimate, release 3 -- 1992-2015
    date:           02-May-2017
    Conventions:    CF-1.6
    _FillValue:     nan
    missing_value:  nan

#### 1. Dimensions
`Dimensions:   (i: 90, j: 90, tile: 13, time: 288)`

The *Dimensions* list now inculdes a new **tile** dimension of 13.

#### 2. Coordinates
```
Coordinates:
    lon_c     (tile, j, i) float64 -111.6 -111.3 -110.9 -110.5 -110.0 -109.3 ...
    lat_c     (tile, j, i) float64 -88.24 -88.38 -88.52 -88.66 -88.8 -88.94 ...
  * tile      (tile) int64 1 2 3 4 5 6 7 8 9 10 11 12 13
``` 

The **tile** coordinate now appears as an array: 1 .. 13 and **lon_c** and **lat_c** have a new **tile** dimension.


#### 3. Data Variables
```
Data variables:
    SSH       (time, tile, j, i) float64 nan nan nan nan nan nan nan nan nan ...
```

The $SSH$ *Data variable* now has a new **tile** dimension.

## Summary

Now you know how to load all 13 NetCDF tile files for both grid parameters and state estimate output variables using `load_all_tiles_from_netcdf`.  All one needs to do is specify whether the file is a grid parameter file or from a $c$, $u$, $v$, or $g$ point and the correct coordinates and dimensional labels will be applied automatically.